In [2]:
!pip install pyspark

  Using cached pyspark-3.4.1.tar.gz (310.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285432 sha256=9aaff74348639424303dba1881e98eecfd934c3d93f35d950e0e558a82c1ec70
  Stored in directory: c:\users\14184\appdata\local\pip\cache\wheels\53\fe\23\517784b9d9dadfb82c5676e76483422096aa5dc20d4d602213
Successfully built pyspark


In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import *

In [2]:
# Start Spark Session
spark = SparkSession.builder.getOrCreate()

In [29]:
df = spark.read.option("header", True).option('inferSchema',True).csv("C:\My data\Portfolio 00\Python\Auction Project PySpark\ipl_2022_dataset.csv")

In [30]:
df.show(5)

+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
|_c0|         Player|Base Price|       TYPE|COST IN ₹ (CR.)|Cost IN $ (000)|2021 Squad|          Team|
+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
|  0|    Rashid Khan|Draft Pick|     BOWLER|           15.0|         1950.0|       SRH|Gujarat Titans|
|  1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|           15.0|         1950.0|        MI|Gujarat Titans|
|  2|Lockie Ferguson|      2 Cr|     BOWLER|           10.0|         1300.0|       KKR|Gujarat Titans|
|  3|  Rahul Tewatia|   40 Lakh|ALL-ROUNDER|            9.0|         1170.0|        RR|Gujarat Titans|
|  4|   Shubman Gill|Draft Pick|     BATTER|            8.0|         1040.0|       KKR|Gujarat Titans|
+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
only showing top 5 rows



In [31]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Base Price: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- COST IN ₹ (CR.): double (nullable = true)
 |-- Cost IN $ (000): double (nullable = true)
 |-- 2021 Squad: string (nullable = true)
 |-- Team: string (nullable = true)



In [32]:
# Total numbers of rows and columns

print('Rows: ', df.count())

print('Columns: ', len(df.columns))

Rows:  633
Columns:  8


In [33]:
# Unique Values in 'Base Price' Columns

df.select('Base Price').distinct().collect()

[Row(Base Price='20 Lakh'),
 Row(Base Price='2 Cr'),
 Row(Base Price='Draft Pick'),
 Row(Base Price='40 Lakh'),
 Row(Base Price='30 Lakh'),
 Row(Base Price='1 Cr'),
 Row(Base Price='50 Lakh'),
 Row(Base Price='1.5 Cr'),
 Row(Base Price='75 Lakh'),
 Row(Base Price='Retained')]

In [34]:
# Distribution of values inside column 'Base Price'

df.groupBy('Base Price').count().show()

+----------+-----+
|Base Price|count|
+----------+-----+
|   20 Lakh|  344|
|      2 Cr|   48|
|Draft Pick|    6|
|   40 Lakh|   16|
|   30 Lakh|    9|
|      1 Cr|   33|
|   50 Lakh|  104|
|    1.5 Cr|   20|
|   75 Lakh|   26|
|  Retained|   27|
+----------+-----+



In [35]:
# Rename columns and save it in the variable df2

df2 = df.withColumnRenamed("Player", "player")\
      .withColumnRenamed("Base Price", "base_price")\
      .withColumnRenamed("Type", "type")\
      .withColumnRenamed("COST IN ₹ (CR.)", "cost_inr")\
      .withColumnRenamed("Cost IN $ (000)", "cost_usd")\
      .withColumnRenamed("2021 Squad", "2021_team")\
      .withColumnRenamed("Team", "2022_team")

In [36]:
df2.show(5)

+---+---------------+----------+-----------+--------+--------+---------+--------------+
|_c0|         player|base_price|       type|cost_inr|cost_usd|2021_team|     2022_team|
+---+---------------+----------+-----------+--------+--------+---------+--------------+
|  0|    Rashid Khan|Draft Pick|     BOWLER|    15.0|  1950.0|      SRH|Gujarat Titans|
|  1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|    15.0|  1950.0|       MI|Gujarat Titans|
|  2|Lockie Ferguson|      2 Cr|     BOWLER|    10.0|  1300.0|      KKR|Gujarat Titans|
|  3|  Rahul Tewatia|   40 Lakh|ALL-ROUNDER|     9.0|  1170.0|       RR|Gujarat Titans|
|  4|   Shubman Gill|Draft Pick|     BATTER|     8.0|  1040.0|      KKR|Gujarat Titans|
+---+---------------+----------+-----------+--------+--------+---------+--------------+
only showing top 5 rows



In [37]:
# Dropping USD Column

df3 = df2.drop('cost_usd')

In [38]:
df3.show(5)

+---+---------------+----------+-----------+--------+---------+--------------+
|_c0|         player|base_price|       type|cost_inr|2021_team|     2022_team|
+---+---------------+----------+-----------+--------+---------+--------------+
|  0|    Rashid Khan|Draft Pick|     BOWLER|    15.0|      SRH|Gujarat Titans|
|  1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|    15.0|       MI|Gujarat Titans|
|  2|Lockie Ferguson|      2 Cr|     BOWLER|    10.0|      KKR|Gujarat Titans|
|  3|  Rahul Tewatia|   40 Lakh|ALL-ROUNDER|     9.0|       RR|Gujarat Titans|
|  4|   Shubman Gill|Draft Pick|     BATTER|     8.0|      KKR|Gujarat Titans|
+---+---------------+----------+-----------+--------+---------+--------------+
only showing top 5 rows



In [45]:
#Name top 3 players who got paid the most

df3.select('player','cost_inr').filter(df3['type'] != 'BATTER').orderBy('cost_inr', ascending = False).show(3)

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|       KL Rahul|    17.0|
|Ravindra Jadeja|    16.0|
|   Rishabh Pant|    16.0|
+---------------+--------+
only showing top 3 rows



In [48]:
df3.createOrReplaceTempView('ipl')

In [52]:
spark.sql("""SELECT player, cost_inr FROM ipl WHERE type = 'BATTER' ORDER BY 2 DESC LIMIT 3""").show()

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|   Rohit Sharma|    16.0|
|    Virat Kohli|    15.0|
|Kane Williamson|    14.0|
+---------------+--------+



In [55]:
# Name top 5 bowlers who get paid the most

df3.select('player','cost_inr').filter(df3.type == 'BOWLER').orderBy('cost_inr',ascending = False).show(5)

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|    Rashid Khan|    15.0|
|  Deepak Chahar|    14.0|
| Jasprit Bumrah|    12.0|
| Shardul Thakur|   10.75|
|Lockie Ferguson|    10.0|
+---------------+--------+
only showing top 5 rows



In [56]:
spark.sql("""SELECT player, cost_inr FROM ipl WHERE type = 'BOWLER' ORDER BY 2 DESC LIMIT 5""").show()

# Here sort the result in database by name, so the player is different

+--------------+--------+
|        player|cost_inr|
+--------------+--------+
|   Rashid Khan|    15.0|
| Deepak Chahar|    14.0|
|Jasprit Bumrah|    12.0|
|Shardul Thakur|   10.75|
|    Avesh Khan|    10.0|
+--------------+--------+



In [59]:
# Name 5 lowest 5 paid wicket-keeper

df3.select('player','cost_inr').filter((df3.type == 'WICKETKEEPER') & (df3.cost_inr.isNotNull())).orderBy('cost_inr',ascending = True).show(5)

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|  Jitesh Sharma|     0.2|
|    Aryan Juyal|     0.2|
|  N. Jagadeesan|     0.2|
|Luvnith Sisodia|     0.2|
| Baba Indrajith|     0.2|
+---------------+--------+
only showing top 5 rows



In [61]:
spark.sql("""SELECT player, cost_inr FROM ipl WHERE type = 'WICKETKEEPER' AND cost_inr is not null ORDER BY 2 LIMIT 5""").show()

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|    Dhruv Jurel|     0.2|
|  Jitesh Sharma|     0.2|
|  N. Jagadeesan|     0.2|
|    Aryan Juyal|     0.2|
|Luvnith Sisodia|     0.2|
+---------------+--------+



In [67]:
# What is the average pay for batsman, bowler, all-rounder, wicket-keeper?

from pyspark.sql import functions as F

df3.groupBy('type').agg(F.round(F.mean('cost_inr'),3).alias('average_price')).show()

+------------+-------------+
|        type|average_price|
+------------+-------------+
|WICKETKEEPER|        5.087|
|      BOWLER|        3.069|
| ALL-ROUNDER|        3.606|
|      BATTER|        4.108|
+------------+-------------+



In [65]:
spark.sql("""SELECT type, round(avg(cost_inr),2) average_price FROM ipl GROUP BY 1 ORDER BY 2 DESC""").show()

+------------+-------------+
|        type|average_price|
+------------+-------------+
|WICKETKEEPER|         5.09|
|      BATTER|         4.11|
| ALL-ROUNDER|         3.61|
|      BOWLER|         3.07|
+------------+-------------+



In [70]:
# List of Retained players with team name and salary

df3.select('player','2021_team','cost_inr').where(df3.base_price == 'Retained').orderBy('cost_inr',ascending = False).show(10)

+---------------+---------+--------+
|         player|2021_team|cost_inr|
+---------------+---------+--------+
|Ravindra Jadeja|      CSK|    16.0|
|   Rishabh Pant|       DC|    16.0|
|   Rohit Sharma|       MI|    16.0|
|    Virat Kohli|      RCB|    15.0|
|   Sanju Samson|       RR|    14.0|
|Kane Williamson|      SRH|    14.0|
|       MS Dhoni|      CSK|    12.0|
|  Andre Russell|      KKR|    12.0|
| Mayank Agarwal|     PBKS|    12.0|
| Jasprit Bumrah|       MI|    12.0|
+---------------+---------+--------+
only showing top 10 rows



In [71]:
spark.sql("""SELECT player, cost_inr FROM ipl WHERE base_price = 'Retained' ORDER BY 2 DESC LIMIT 5""").show()

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|Ravindra Jadeja|    16.0|
|   Rishabh Pant|    16.0|
|   Rohit Sharma|    16.0|
|    Virat Kohli|    15.0|
|   Sanju Samson|    14.0|
+---------------+--------+

